In [ ]:
# ================================================================
# 🧪 Lab 18: Agent-to-Agent Protocol (AAP) using NVIDIA NIM
# Objective: Demonstrate Planner → Executor → Reviewer workflow
# Model: meta/llama3-8b-instruct via NVIDIA NIM API
# ================================================================

import os, json, requests, time

# === Step 1: Setup & Configuration ===
os.environ['NVIDIA_API_KEY'] = ''   # 🔒 Replace with your actual key
NIM_URL = "https://integrate.api.nvidia.com/v1/chat/completions"
MODEL = "meta/llama3-8b-instruct"

print("✅ NVIDIA NIM configured successfully.")
print("Endpoint:", NIM_URL)


✅ NVIDIA NIM configured successfully.
Endpoint: https://integrate.api.nvidia.com/v1/chat/completions


In [3]:
# === Step 2: Define Helper Function for NVIDIA NIM Calls ===

def call_nvidia(prompt, max_tokens=100, temperature=0.5):
    """Send prompt to NVIDIA NIM and return model response text."""
    headers = {
        "Authorization": f"Bearer {os.environ['NVIDIA_API_KEY']}",
        "Accept": "application/json",
        "Content-Type": "application/json"
    }

    payload = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": 0.7
    }

    print(f"\n📡 Sending prompt to NVIDIA NIM:\n➡️ {prompt}\n")
    response = requests.post(NIM_URL, headers=headers, json=payload)
    
    if response.status_code != 200:
        print("❌ Error:", response.status_code, response.text)
        return None

    try:
        data = response.json()
        output = data["choices"][0]["message"]["content"]
        print("🧠 Model Response:\n", output, "\n")
        return output
    except Exception as e:
        print("⚠️ Failed to parse response:", e)
        print(response.text)
        return None


In [4]:
# === Step 3: Define Agents (Planner, Executor, Reviewer) ===

class PlannerAgent:
    def create_plan(self, goal):
        prompt = f"Break this goal into 2-3 clear tasks:\nGoal: {goal}"
        plan = call_nvidia(prompt)
        return {"type": "create_task", "goal": goal, "plan": plan}

class ExecutorAgent:
    def execute_tasks(self, plan):
        prompt = f"Execute each of these tasks and provide concise results:\n{plan}"
        result = call_nvidia(prompt)
        return {"type": "execute_step", "result": result}

class ReviewerAgent:
    def review_results(self, execution_result):
        prompt = f"Review the following results for quality and correctness. Summarize them:\n{execution_result}"
        review = call_nvidia(prompt)
        return {"type": "step_result", "review": review}


In [5]:
# === Step 4: Run the AAP Workflow (Planner → Executor → Reviewer) ===

goal = "Explain what an API is and give one real-world example."

print("\n🎯 Goal:", goal)

# Initialize agents
planner = PlannerAgent()
executor = ExecutorAgent()
reviewer = ReviewerAgent()

# Step 1: Planner creates a plan
print("\n🧠 [Planner Agent] Creating task plan...")
plan = planner.create_plan(goal)
print("📋 Planner Output:", json.dumps(plan, indent=2))

# Step 2: Executor performs the tasks
print("\n⚙️ [Executor Agent] Executing plan...")
execution = executor.execute_tasks(plan["plan"])
print("📊 Executor Output:", json.dumps(execution, indent=2))

# Step 3: Reviewer evaluates the results
print("\n✅ [Reviewer Agent] Reviewing execution results...")
review = reviewer.review_results(execution["result"])
print("📘 Reviewer Output:", json.dumps(review, indent=2))



🎯 Goal: Explain what an API is and give one real-world example.

🧠 [Planner Agent] Creating task plan...

📡 Sending prompt to NVIDIA NIM:
➡️ Break this goal into 2-3 clear tasks:
Goal: Explain what an API is and give one real-world example.

🧠 Model Response:
 Here are 2-3 clear tasks to help you achieve your goal:

**Task 1: Define what an API is**

* Research and write a clear definition of what an API (Application Programming Interface) is, including its purpose and how it works.
* Ensure your definition is concise and easy to understand for a general audience.

**Task 2: Provide a real-world example of an API**

* Choose a well-known example of an API in use, such as the Twitter API or the 

📋 Planner Output: {
  "type": "create_task",
  "goal": "Explain what an API is and give one real-world example.",
  "plan": "Here are 2-3 clear tasks to help you achieve your goal:\n\n**Task 1: Define what an API is**\n\n* Research and write a clear definition of what an API (Application Progr